# 1.3 -  Anwendungsbeispiel

<img style="float: right; margin:5px 0px 0px 10px" src="img/audio.jpg" width="400">


## Inhalt  

<table style="width:256px; border: 1px solid black; display: inline-block">
  <tr>
    <td  style="text-align:right" width=64px><img src="img/IMG-Python.png" style="float:left"></td>
      <td style="text-align:left" width=256px>
          <a style="color:black; font-size:14px; font-weight:bold; text-decoration:none" href='#intro'>1. Audiosignale in Python</a>
    </td>
  </tr>  
    <tr>
    <td  style="text-align:right" width=64px><img src="img/IMG-fir.jpg" style="float:left"></td>
      <td style="text-align:left" width=256px>
          <a style="color:black; font-size:14px; font-weight:bold; text-decoration:none" href='#fir'>2. FIR-Filterung</a>
    </td>
  </tr>  
    <tr>
    <td  style="text-align:right" width=64px><img src="img/IMG-iir.jpg" style="float:left"></td>
      <td style="text-align:left" width=256px>
          <a style="color:black; font-size:14px; font-weight:bold; text-decoration:none" href='#iir'>3. IIR-Filterung</a>
    </td>
  </tr>
</table>



---- 

Der Zweck eines Filters besteht darin, ein Eingangssignal durch eine bestimmte Operation in ein Ausgangssignal umzuwandeln, nämlich Filterung. Es gibt zwei Möglichkeiten einen digitalen Filter zu implementieren: 
- Softwareprogrammierung  
- Spezielle Hardware oder allgemeiner digitaler Signalprozessor (DSP)  

Für den Filterentwurf wirken sich unterschiedliche Strukturen auf die Berechnungskomplexität, den Berechnungsfehler und die Stabilität usw. aus. Daher sollte die Filterleistung für bestimmte Anwendungen angemessen berücksichtigt werden, um die Implementierung zu vereinfachen.   

In Bezug auf die Leistung umfasst die IIR-Filterübertragungsfunktion zwei Sätze einstellbarer Faktoren: Null- und Polstellen. Die einzige Einschränkung für die Polstellen sind, dass alle im Einheitskreis liegen müssen. Im Vergleich zu FIR-Filtern können meist niedrigere Ordnungen verwendet werden, um dieselbe Selektivität zu erhalten, was zu einer kleineren Anzahl an Speichereinheiten und ein geringerer Rechenaufwand, was insgesamt die Effizienz erhöht. Dieser höhere Wirkungsgrad geht jedoch zu Lasten der Nichtlinearität der Phase. Je besser die Selektivität ist, desto schwerwiegender ist die Nichtlinearität der Phase. Der Pol der FIR-Filterübertragungsfunktion ist am Ursprung fixiert und kann seine Leistung nur durch Änderung der Nullposition ändern. Um eine hohe Selektivität zu erreichen, muss daher eine höhere Ordnung verwendet werden. Für denselben Filterdesignindex kann die erforderliche Ordnung des FIR-Filters vielmal höher sein als die des IIR-Filters, so dass die Kosten höher und die Signalverzögerung ist auch größer.

In diesem Notebook werden bestimmte Frequenzkomponenten eines Eingangssignals durch verschiedene Filter entfernt, um die Eigenschaften zu vergleichen. Genauer gesagt, wir werden Bandsperrfilter zum Entfernen mit unterschiedlichen Methoden implementieren und die Ergebnisse vergleichen. Die originalen Daten des Audiosignals (akustik.wav) finden Sie unter dem Ordner 'data'.

<a id='intro'></a><div><img src="img/IMG-Python.png" style="float:left"><h2 style="position: relative; top: 6px; left: 6px">1. Audiosignale in Python</h2></div>

#### 1.1 Import von Audiodateien

Um Audiodateien zu bearbeiten, müssen wir sie zuerst in das Projekt einlesen. Dazu gibt es mehrere Methoden, von denen drei als Beispiele nun aufgeführt sind:

```python
path = 'data/akustik.wav'

# 1. wavfile:
from scipy.io import wavfile
sr, audio = wavfile.read(path)

# 2. wave:
import wave
wf = wave.open(path,'rb')
audio = wf.readframes(1024)

# 3. librosa:
import librosa
audio, sr = librosa.load(path)
```

In diesem Notebook verwenden wir die erste Option [`wavfile()`](https://docs.scipy.org/doc/scipy/reference/io.html?highlight=wavfile#module-scipy.io.wavfile), da es schon in dem Modul `scipy` integriert ist und somit installiert ist. Schauen Sie sich nun die Audioaufnahme `data/akustik.wav` im Zeit- und Frequenzbereich an. Importieren Sie dafür zuerst die dafür notwendigen Module:

In [ ]:
# Importieren Sie aus scipy die Teilbibliothek "fftpack" 
# Importieren Sie aus  scipy.io die Teilbibliothek "wavfile"
# Importieren Sie aus matplotlib die Teilbibliothek "pyplot" mit dem Alias "plt"
# Importieren Sie das numpy-Modul mit dem Alias "np"

[..]
[..]
[..]
[..]

In [ ]:
# Lösung
# Importieren Sie aus scipy die Teilbibliothek "fftpack"
# Importieren Sie aus  scipy.io die Teilbibliothek "wavfile"
# Importieren Sie aus matplotlib die Teilbibliothek "pyplot" mit dem Alias "plt"
# Importieren Sie das numpy-Modul mit dem Alias "np"

import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack
from scipy.io import wavfile

In [ ]:
'''
Beispiel: Audiodatei einlesen und visualisieren
'''

# Datei einlesen 
sr, audio = [..]  # ToDo: Lesen Sie Sample Rate und das Audiosignal ein 
audio = audio/np.max(np.abs(audio))   # Normalisierung des Signals

# Initiale Daten
fs_Hz = sr  # Abtrastfrequenz
L = len(audio)  # Länge des Audiosignals
T = L/fs_Hz - 1/fs_Hz  # Zeit
t = np.linspace(0, T, L)  # Zeitbereich
f_Hz = np.linspace(0, fs_Hz/2, int(L/2))  # Frequenzbereich

# Spectrale
audio_fft = [..]   # ToDo: Führen Sie eine FFT des Audiosignals durch


# plot
plt.subplot(121)
plt.title('Audiosignal im Zeitbereich')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, audio)

plt.subplot(122)
plt.title('Audiosignal im Frequenzbereich')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 200)
plt.plot(f_Hz, np.abs(audio_fft[:int(L/2)]))

plt.gcf().set_size_inches(15, 5)
plt.show()

In [ ]:

'''
Beispiel: Audiodatei einlesen und visualisieren
'''

# Datei einlesen 
sr, audio = wavfile.read('data/akustik.wav')  # Sample Rate, Audiosignal im Array-Form
audio = audio/np.max(np.abs(audio))   # Normalisierung

# Initiale Daten
fs_Hz = sr  # Abtrastfrequenz
L = len(audio)  # Länge des Audiosignals
T = L/fs_Hz - 1/fs_Hz  # Zeit
t = np.linspace(0, T, L)  # Zeitbereich
f_Hz = np.linspace(0, fs_Hz/2, int(L/2))  # Frequenzbereich

# Spectrale
audio_fft = fftpack.fft(audio)

# plot
plt.subplot(121)
plt.title('Audiosignal im Zeitbereich')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, audio)
plt.subplot(122)
plt.title('Audiosignal im Frequenzbereich')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 200)
plt.plot(f_Hz, np.abs(audio_fft[:int(L/2)]))
plt.gcf().set_size_inches(15, 5)
plt.show()

#### 1.2 Abspielen von Audio

Die Ausgabe von Audiosignalen kann in Python auch mit unterschiedlichen externen Modulen realisiert werden. Falls Sie das Modul nicht installiert haben, können Sie dies via `pip install "module"` durchführen. Hier werden Ihnen drei mögliche Module mit Ihren individuellen Vorzügen vorgestellt:  

- [`playsound`](https://pypi.org/project/playsound/)   
Das PlaySound-Modul ist ein plattformübergreifendes Modul, das Audiodateien abspielen kann.

In [ ]:
from playsound import playsound

In [ ]:
playsound('data/akustik.wav')

- [`IPython.display`](https://ipython.org/ipython-doc/dev/api/generated/IPython.display.html)  
Mit diesem Modul wird die Audiodatei mit einem Audioplayer geöffnet, wodurch sich 

In [ ]:
import IPython.display as ipd

In [ ]:
ipd.Audio('data/akustik.wav')

- [`simpleaudio`](https://simpleaudio.readthedocs.io/en/latest/)  
Das Modul ermöglicht es, sowohl wav-Dateien als auch NumPy-Arrays abgespielt zu können. Diese Eigenschaft ist für dieses Notebook sehr nützlich, weshalb im Weiteren auf `simpleaudio` zurückgegriffen wird.

In [ ]:
import simpleaudio as sa

In [ ]:
# Beispiel: wav-Dateien abspielen

# Datei einlesen
wave_obj = sa.WaveObject.from_wave_file('data/akustik.wav')

# abspielen
play_obj = wave_obj.play()
play_obj.wait_done()

In [ ]:
# Beispiel: NumPy-Arrays (Audiosignal) abspielen

# Datei ablesen 
sr, audio = wavfile.read('data/akustik.wav')

# abspielen
play_obj = sa.play_buffer(audio, 1, 2, sr)
play_obj.wait_done()

In [ ]:
# Beispiel: NumPy-Arrays (Sinussignal) abspielen

# Sinussignal erzeugen
t = np.linspace(0, 3, 8000)
sine = np.sin(440 * np.pi * t)

# Werte im 16-Bit-Bereich konvertieren
sound = sine * (2**15 - 1) / np.max(sine)
sound = sound.astype(np.int16)

# abspielen
play_obj = sa.play_buffer(sound, 1, 2, 8000)
play_obj.wait_done()

#### 1.3 Signale bearbeiten / Störsignal erstellen

Wenn das Audiosignal als numpy-Array vorliegt, kann das Signal sehr leicht bearbeitet werden. Zuerst soll dafür auf die Audiodatei `akustik.wav` ein Sinus mit der Frequenz von 440 Hz addiert werden. Dadurch erhält die Audiodatei ein Störsignal, welches Sie danach wieder davon zu entfernen versuchen.

In [ ]:
# Lösung
# Importieren Sie aus scipy die Teilbibliothek "signal" 
[..]

In [ ]:
# Importieren Sie aus scipy die Teilbibliothek "signal" 
from scipy import signal

In [ ]:
'''
Beispiel: Signal addieren, visualisieren und abspielen
'''

# Datei einlesen (originales Audiosignal)
[..]   # ToDo: lesen sie die Audiodatei "akustik.wav" mit dem Objekt wavfile.read() ein
[..]   # ToDo: Normalisieren Sie das numpy-Array

# Initialisierung der Variablen
fs_Hz = [..]     # ToDo: Bestimmen Sie die Abtastfrequenz
fsin_Hz = [..]   # ToDo: Bestimmen Sie die Sinusfrequenz
L = [..]         # ToDo: Bestimmen Sie die Länge des Audio-Arrays
T = [..]         # ToDo: Bestimmen Sie die Zeitdauer der Aufnahme
t = [..]         # ToDo: Erzeugen Sie das zeitliche Array (nutzen Sie dafür np.linspace())
f_Hz = [..]      # ToDo: Erzeugen Sie das Array für den Frequenzbereich (nutzen Sie dafür np.linspace())

# Sinussignale erstellen
sine = [..]      # ToDo: Erzeugen Sie das Sinussignal mit einer Frequenz von 440 Hz im Zeitbereich
sine_fft = [..]  # ToDo: Berechnen Sie mittels FFT den Frequenzbereich des Signals

# Graphische Darstellung
plt.subplot(211)
plt.title('Störsignal im Zeitbereich')
plt.xlabel('Zeit [s]') 
plt.xlim(0, 0.1)
plt.ylabel('Amplitude') 
plt.ylim(-1, 1)
plt.plot(t, sine)

plt.subplot(222)
plt.title('Störsignal im Frequenzbereich')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 500)
plt.plot(f_Hz, np.abs(sine_fft[:int(L/2)]))

plt.gcf().set_size_inches(15, 5)
plt.show()

In [ ]:
# Signal zur Filterung
s = [..]         # ToDo: Addieren Sie das Audiosignal mit dem Sinussignal (im Zeitbereich)
S = [..]         # ToDo: Berechnen Sie mittels FFT den Frequenzbereich des Signals mit Störung

# ToDo: Abspielen des erzeugten Signals


# Fensterung (optional)
# ToDo: Erzeugen Sie ein Hanning-Fenster der Länge "L" mit signal.get_window()
# ToDo: Wenden Sie das Fenster auf das Signal "s" an.

plt.subplot(211)
plt.title('Audiosignal mit Störung im Zeitbereich')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, s)

plt.subplot(222)
plt.title('Audiosignal mit Störung im Frequenzbereich')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 500)
plt.plot(f_Hz, np.abs(S[:int(L/2)]))

plt.gcf().set_size_inches(15, 5)
plt.show()


In [ ]:
# Lösung
'''
Beispiel: Signal addieren, visualisieren und abspielen
'''

# Datei ablesen (originales Audiosignal)
sr, audio = wavfile.read('data/akustik.wav')
audio = audio/np.max(np.abs(audio)) 

# Initiale Daten
fs_Hz = sr  # Abtrastfrequenz
fsin_Hz = 440  # Sinusfrequenz
L = len(audio)  # Länge des Audiosignals
T = L/fs_Hz - 1/fs_Hz  # Zeitraum
t = np.linspace(0, T, L)  # Zeitbereich
f_Hz = np.linspace(0, fs_Hz/2, int(L/2))  # Frequenzbereich

# Sinussignale erstellen
sine = 1/2 * np.sin(fsin_Hz * 2*np.pi * t)
sine_fft = fftpack.fft(sine)

# Signal zur Filterung
s = audio + sine  # Additives Signal
S = fftpack.fft(s)  # Frequenzgang

# Fensterung (optional)
wd = signal.get_window('hanning', L)
s_windowed = s * wd

# plot
plt.subplot(221)
plt.title('Störsignal im Zeitbereich')
plt.xlabel('Zeit [s]') 
plt.xlim(0, 0.1)
plt.ylabel('Amplitude') 
plt.ylim(-1, 1)
plt.plot(t, sine)
plt.subplot(222)
plt.title('Störsignal im Frequenzbereich')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 500)
plt.plot(f_Hz, np.abs(sine_fft[:int(L/2)]))
plt.subplot(223)
plt.title('Audiosignal mit Störung im Zeitbereich')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, s)
plt.subplot(224)
plt.title('Audiosignal mit Störung im Frequenzbereich')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 500)
plt.plot(f_Hz, np.abs(S[:int(L/2)]))
plt.gcf().set_size_inches(15, 10)
plt.show()


# Abspielen
sound = (s * (2**15 - 1) / np.max(np.abs(s))).astype(np.int16)
play_obj = sa.play_buffer(sound, 1, 2, fs_Hz)
play_obj.wait_done()

----

<a id='fir'></a><div><img src="img/IMG-fir.jpg" style="float:left"><h2 style="position: relative; top: 6px; left: 6px">2. FIR-Filterung </h2></div>

#### 2.1 FIR-Filter mittels IDFT

Jetzt entwerfen wir einen Bandsperrfilter mittels IDFT, um das Sinussignal von Audiosignal zu entfernen. Der Filter soll dabei eine Bandbreite von 30 Hz haben.

In [ ]:
'''
Beispiel: Entfernen von Signalen mittels selbst definierten FIR-Bandsperrfilter
'''

# Filter
bw = 30  # Bandbreite
H = [..]         # ToDo: Erzeugen Sie mit np.where einen idealen Bandsperrfilter mit Bandbreite 30 Hz
h = [..]         # ToDo: Berechnen Sie die Impulsantwort mittels ifft (ifft-shift nicht vergessen!)

# Filterung
s_filtered = [..] # ToDo: Falten Sie das gestörte Signal "s" (oder "s_windowed") mit dem Filter "h" 
S_filtered = [..] # ToDo: Berechnen Sie den Frequenzgang von s_filtered mittels FFT

# Plot
plt.subplot(221)
plt.title('Impulsantwort des Filters')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, np.real(h))
plt.subplot(222)
plt.title('Spectrale des Bandsperrfilters')
plt.xlabel('Frequenz [rad]') 
#plt.xlim(0, 1000)
plt.ylabel('Amplitude') 
plt.plot(f_Hz, H)
plt.subplot(223)
plt.title('Audiosignal nach Bandsperrfilterung')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, np.real(s_filtered))
plt.subplot(224)
plt.title('Frequenzgang nach Filterung')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 200)
plt.plot(f_Hz, np.abs(S_filtered[:int(L/2)]))
plt.gcf().set_size_inches(15, 10)
plt.show()



In [ ]:
# Abspielen des gefilterten Signals
sound = [..]    # ToDo: Konvertieren sie das Audiosignal in 16-Bit-Format (INT16) 
play_obj = [..] # ToDo: Abspielen von sound

In [ ]:
# Lösung

'''
Beispiel: Entfernen von Signalen mittels selbst definierten FIR-Bandsperrfilters 
'''

# Filter
bw = 30  # Bandbreite
H = np.where((f_Hz < (fsin_Hz+bw)) & (f_Hz > (fsin_Hz-bw)), 0, 1)  
h = np.fft.ifftshift(fftpack.ifft(H, L))   

# Filterung
s_filtered = signal.convolve(s_windowed, h, 'same')  # Faltung
S_filtered = fftpack.fft(s_filtered)

# Plot
plt.subplot(221)
plt.title('Impulsantwort des Filters')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, np.real(h))
plt.subplot(222)
plt.title('Spectrale des Bandsperrfilters')
plt.xlabel('Frequenz [rad]') 
#plt.xlim(0, 1000)
plt.ylabel('Amplitude') 
plt.plot(f_Hz, H)
plt.subplot(223)
plt.title('Audiosignal nach Bandsperrfilterung')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, np.real(s_filtered))
plt.subplot(224)
plt.title('Frequenzgang nach Filterung')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 200)
plt.plot(f_Hz, np.abs(S_filtered[:int(L/2)]))
plt.gcf().set_size_inches(15, 10)
plt.show()

# Abspielen
sound = (np.real(s_filtered) * (2**15 - 1) / np.max(np.abs(s_filtered))).astype(np.int16)
play_obj = sa.play_buffer(sound, 1, 2, fs_Hz)

#### 2.2 FIR-Filter mittels __firwin()__  
Für solche Anwendungen können wir aber auch die vorhandenen Filter wie [`signal.firwin()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.firwin.html) in __[`scipy.signal`](https://docs.scipy.org/doc/scipy/reference/signal.html)__ direkt benutzen. Das Ergebnis ist ähnlich:

In [ ]:
'''
Aufgabe: Entfernen von Frequenzen mittels signal.firwin() 
'''

# zu verwendende Parameter für firwin()
bw = 30  # Bandbreite
n = 5001  # Ordnung
wn = [(fsin_Hz-bw), (fsin_Hz+bw)]

# Filterung mit Fenster
h_fir = [..]       # ToDo: Erzeugen Sie mit signal.firwin() einen Bandsperrfilter.
H_fir = [..]       # ToDo: Berechnen Sie mittels FFT den Frequenzbereich des Bandpassfilters
s_filtered = [..]  # ToDo: Falten Sie das gestörte Signal "s" mit dem Filter "h_filtered" 
S_filtered = [..]  # ToDo: Berechnen Sie mittels FFT den Frequenzbereich des gefilterten Signals

# plot
plt.subplot(221)
plt.title('Impulsantwort des Filters')
plt.xlabel('Sample Nummer') 
plt.ylabel('Amplitude') 
plt.plot(h_fir)

plt.subplot(222)
plt.title('Spektral des Filters')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.plot(f_Hz, np.abs(H_fir[:int(L/2)]))

plt.subplot(223)
plt.title('Audiosignal nach Bandsperrfilterung')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, s_filtered)

plt.subplot(224)
plt.title('Frequenzgang nach Filterung')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 200)
plt.plot(f_Hz, np.abs(S_filtered[:int(L/2)]))
plt.gcf().set_size_inches(15, 10)
plt.show()

In [ ]:
# Abspielen des gefilterten Signals
sound = [..]    # ToDo: Konvertieren sie das Audiosignal in 16-Bit-Format (INT16) 
play_obj = [..] # ToDo: Abspielen von sound

In [ ]:
# Lösung

'''
Aufgabe: Entfernen von Signalen mittels signal.firwin() 
'''
# Parameter
bw = 30  # Bandbreite
n = 5001  # Ordnung
wn = [(fsin_Hz-bw), (fsin_Hz+bw)]

# Filterung mit Fenster
h_fir = signal.firwin(n, wn, window='hamming', fs=fs_Hz)
H_fir = fftpack.fft(h_fir, L)  # Spektral des Filters
s_filtered = signal.convolve(s, h_fir, 'same')  # Faltung
S_filtered = fftpack.fft(s_filtered)  # Spektral des Signals

# plot
plt.subplot(221)
plt.title('Impulsantwort des Filters')
plt.xlabel('Sample Nummer') 
plt.ylabel('Amplitude') 
plt.plot(h_fir)
plt.subplot(222)
plt.title('Spektral des Filters')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.plot(f_Hz, np.abs(H_fir[:int(L/2)]))
plt.subplot(223)
plt.title('Audiosignal nach Bandsperrfilterung')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, s_filtered)
plt.subplot(224)
plt.title('Frequenzgang nach Filterung')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 200)
plt.plot(f_Hz, np.abs(S_filtered[:int(L/2)]))
plt.gcf().set_size_inches(15, 10)
plt.show()

# Abspielen
sound = (s_filtered * (2**15 - 1) / np.max(np.abs(s_filtered))).astype(np.int16)
play_obj = sa.play_buffer(sound, 1, 2, fs_Hz)

Vom Ergebnis können wir finden, um das Sinussignal zu entfernen, braucht man eine sehr hohe Ordnung zu verwenden, was bedeutet die große Anforderung von Speicherplatz, einen goßen Rechenaufwand und niedrige Effizienz.  

Anschließend verwenden wir IIR-Filter zum Entfernen von Signalen.

----

<a id='iir'></a><div><img src="img/IMG-iir.jpg" style="float:left"><h2 style="position: relative; top: 6px; left: 6px">3. IIR-Filterung </h2></div>

#### 3.1 Butterworth Filter

- Butterworth Filterentwurf mit Ordnung=10

In [ ]:
'''
Beispiel: Entfernen von Signalen mittels IIR-Bandsperrfilters (Butterworth)
'''
# Parameter
bw = 30  # Bandbreite des Filters
n = 10  # Ordnung des IIR-Filters
wn = [fsin_Hz-bw, fsin_Hz+bw]  # Frequenzbereich des Sperrbands

# Filterung 
sos = signal.butter(n, wn, 'bs', analog=False, fs=fs_Hz, output='sos')
w, H_butt = signal.sosfreqz(sos, int(L/2), fs=fs_Hz)
s_filtered = signal.sosfilt(sos, s_windowed)
S_filtered = fftpack.fft(s_filtered)

# plot
plt.subplot(311)
plt.title('Frequenzgang des Butterworth Filters mit Ordnung=%d' %n)
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.plot(f_Hz, np.abs(H_butt[:int(L/2)]))
plt.subplot(312)
plt.title('Audiosignal nach Filterung')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, s_filtered)
plt.subplot(313)
plt.title('Frequenzgang nach Filterung')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 200)
plt.plot(f_Hz, np.abs(S_filtered[:int(L/2)]))
plt.gcf().set_size_inches(8, 16)
plt.show()

# Abspielen
sound = (s_filtered * (2**15 - 1) / np.max(np.abs(s_filtered))).astype(np.int16)
play_obj = sa.play_buffer(sound, 1, 2, fs_Hz)

In [ ]:
# Lösung
'''
Beispiel: Entfernen von Signalen mittels IIR-Bandsperrfilters (Butterworth)
'''
# Parameter
bw = 30  # Bandbreite des Filters
n = 10  # Ordnung des IIR-Filters
wn = [fsin_Hz-bw, fsin_Hz+bw]  # Frequenzbereich des Sperrbands

# Filterung 
sos = signal.butter(n, wn, 'bs', analog=False, fs=fs_Hz, output='sos')
w, H_butt = signal.sosfreqz(sos, int(L/2), fs=fs_Hz)
s_filtered = signal.sosfilt(sos, s_windowed)
S_filtered = fftpack.fft(s_filtered)

# plot
plt.subplot(311)
plt.title('Frequenzgang des Butterworth Filters mit Ordnung=%d' %n)
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.plot(f_Hz, np.abs(H_butt[:int(L/2)]))
plt.subplot(312)
plt.title('Audiosignal nach Filterung')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, s_filtered)
plt.subplot(313)
plt.title('Frequenzgang nach Filterung')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 200)
plt.plot(f_Hz, np.abs(S_filtered[:int(L/2)]))
plt.gcf().set_size_inches(8, 16)
plt.show()

# Abspielen
sound = (s_filtered * (2**15 - 1) / np.max(np.abs(s_filtered))).astype(np.int16)
play_obj = sa.play_buffer(sound, 1, 2, fs_Hz)

- Butterworth Filterentwurf mit Ordnungselektion

In [ ]:
'''
Aufgabe: Entfernen von Signalen mittels IIR-Bandsperrfilters (Butterworth)
'''
# Ordnungselektion
bw = 30  # Bandbreite
n, wn = signal.buttord([fsin_Hz-bw-10, fsin_Hz+bw+10], [fsin_Hz-bw, fsin_Hz+bw], 10, 60, False, fs_Hz)

# Filterung 
sos = signal.butter(n, wn, 'bs', False, 'sos', fs_Hz)
w, H_butt = signal.sosfreqz(sos, int(L/2), fs=fs_Hz)
s_filtered = signal.sosfilt(sos, s_windowed)
S_filtered = fftpack.fft(s_filtered)

# plot
plt.subplot(311)
plt.title('Frequenzgang des Butterworth Filters mit Ordnung=%d' %n)
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.plot(f_Hz, np.abs(H_butt[:int(L/2)]))
plt.subplot(312)
plt.title('Audiosignal nach Filterung')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, s_filtered)
plt.subplot(313)
plt.title('Frequenzgang nach Filterung')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 200)
plt.plot(f_Hz, np.abs(S_filtered[:int(L/2)]))
plt.gcf().set_size_inches(8, 16)
plt.show()

# Abspielen
sound = (s_filtered * (2**15 - 1) / np.max(np.abs(s_filtered))).astype(np.int16)
play_obj = sa.play_buffer(sound, 1, 2, fs_Hz)

Für dieselbe Filterqualität ist die Ordnungszahl der IIR-Filter viel geringer!

### 3.2 Chebyshev Filter

In [ ]:
'''
Aufgabe: Entfernen von Signalen mittels IIR-Bandsperrfilters (Chebyshev II mit Ordnungselektion)
'''

# Ordnungselektion
bw = 30  # Bandbreite
n, wn = signal.cheb2ord([fsin_Hz-bw-10, fsin_Hz+bw+10], [fsin_Hz-bw, fsin_Hz+bw], 10, 60, False, fs_Hz)

# Filterung
sos = signal.cheby2(n, 60, wn, 'bs', False, 'sos', fs_Hz)
w, H_cheby = signal.sosfreqz(sos, int(L/2), fs=fs_Hz)
s_filtered = signal.sosfilt(sos, s_windowed)
S_filtered = fftpack.fft(s_filtered)

# plot
plt.subplot(311)
plt.title('Frequenzgang des Chebyshev-II Filters mit Ordnung=%d' %n)
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.plot(f_Hz, np.abs(H_cheby[:int(L/2)]))
plt.subplot(312)
plt.title('Audiosignal nach Filterung')
plt.xlabel('Zeit [s]') 
plt.ylabel('Amplitude') 
plt.plot(t, s_filtered)
plt.subplot(313)
plt.title('Frequenzgang nach Filterung')
plt.xlabel('Frequenz [rad]') 
plt.ylabel('Amplitude') 
plt.ylim(0, 200)
plt.plot(f_Hz, np.abs(S_filtered[:int(L/2)]))
plt.gcf().set_size_inches(8, 16)
plt.show()

# Abspielen
sound = (s_filtered * (2**15 - 1) / np.max(np.abs(s_filtered))).astype(np.int16)
play_obj = sa.play_buffer(sound, 1, 2, fs_Hz)

Zum Schluss können wir das verarbeitete Audiosignal wieder als wav-Datei speichern:

In [ ]:
'''
Beispiel: Daten in eine wav-Datei schreiben
'''
import numpy as np
from scipy.io import wavfile
from playsound import playsound

# Pfad einer neuen wav-Datei bestimmen
file_path = 'data/akustik_filtered.wav'
# Audiodaten in 16-Bit-Format konvertieren
data = (s_filtered * (2**15 - 1) / np.max(np.abs(s_filtered))).astype(np.int16)
# Audiodaten schreiben
wavfile.write(file_path, fs_Hz, data)

# Testen, ob die Daten erfolgreich gespeichert wurde
playsound('data/akustik_filtered.wav')

----

### References

1. Titelbild von [Encarni Mármol](https://lda-audiotech.com/en/2018/10/30/how-to-digitize-analog-audio/)  
2. [Play sound in Python](https://pythonbasics.org/python-play-sound/)